In [1]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import scqubits as scq
import pandas as pd
from matplotlib.animation import FuncAnimation
import scipy.constants as sconst

%matplotlib widget

In [16]:
ratio = 100
Ec = 0.25
Ej = Ec*ratio
N = 10 # number of levels in charge basis
ng = 0.5

T_max = 250
T1 = 50
T2 = 400

P0=0.99
R0 = 2.5 * np.pi

def hamiltonian(Ec, Ej, N, ng):
    """
    Return the charge qubit hamiltonian as a Qobj instance.
    """
    m = np.diag(4 * Ec * (np.arange(-N,N+1)+ng)**2) + 0.5 * Ej * (np.diag(-np.ones(2*N), 1) + 
                                                               np.diag(-np.ones(2*N), -1))
                                                               
    return qt.Qobj(m)

def Hc(Ec, N, ng):
    
    m = np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) 
    
    return qt.Qobj(m)

def Hj(Ej, N, ng):
    
    m = 0.5 * Ej * (np.diag(-np.ones(2*N), 1) + np.diag(-np.ones(2*N), -1))
    
    return qt.Qobj(m)

H_c = Hc(Ec=Ec, N=N, ng = ng)
H_j = Hj(Ej=Ej, N=N, ng=ng)

H_tot = hamiltonian(Ec, Ej, N, ng)
def EJ_tanh_pulse(t, P0, R0, T):
    return (1-P0*(1+np.tanh(-R0+(t/T)*2*R0))/2.0)

def EJ_double_pulse(t, P0, R0, T1, T2):
    return EJ_tanh_pulse(t, P0, R0, T1) + (EJ_tanh_pulse(t - T2, -P0, R0, T1) - 1)

# Time depenedent part (drive) of Hamiltonian



def Hd_coeff(t,args):
    Percentage=args['P0']
    Range=args['R0']
    T1=args['T1']
    T2=args['T2']
    return EJ_double_pulse(t, Percentage, Range, T1, T2)

def dH_dng(Ec, N, ng):
    m = np.diag(8 * Ec * (np.arange(-N,N+1)+ng)) 
    return qt.Qobj(m)

In [25]:
Points=10000
tlist = np.linspace(0, T_max, Points)
H_tot_tdep = [H_c, [H_j, Hd_coeff]]
evals, ekets = H_tot.eigenstates()

charge_noise_op = dH_dng(Ec, N, ng)
gamma_matrix = np.zeros((2*N+1, 2*N+1))
for i in range(2*N+1):
    for j in range(2*N+1):
        gamma_matrix[i, j] = np.abs(charge_noise_op.matrix_element(ekets[i], ekets[j]))**2

qt.Qobj(gamma_matrix[0:7, 0:7])

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = True
Qobj data =
[[6.76284319e-21 6.80988220e+00 2.99893791e-20 4.02519893e-03
  2.57934554e-18]
 [6.80988220e+00 5.70860305e-18 1.30671495e+01 5.37367243e-18
  1.80451894e-02]
 [2.99893888e-20 1.30671495e+01 1.11244350e-15 1.87041052e+01
  2.69027193e-16]
 [4.02519893e-03 5.37367011e-18 1.87041052e+01 8.83353642e-14
  2.36246711e+01]
 [2.57934537e-18 1.80451894e-02 2.69027193e-16 2.36246711e+01
  3.58095100e-12]]

1.1764705882352942e-05